In [ ]:
import numpy as np

def init(rd)
    return basic+coeff*rd     #评估初始适宜程度，随机生成，但有范围，需要生成参数，此处先默认是一次函数的限制
    
def sigma_sd(st,ed,xpos,ypos)。 #用于累计某元胞内成熟蒲公英的数量，如果需要，可以做优化
    cnt=0
    for i in range(st,ed+1):
        cnt+=Grow[xpos][ypos][i]
    return cnt
    
def sigma_p(fx,fy,cx,cy,ti)
    rel_x=fx-cx
    rel_y=fy-cy
    return Wind_prb(rel_x,rel_y,ti)*sigma_sd(st_start,st_end,cx,cy)      #st为成熟阶段，本段算的是从(fx,fy)传到(cx,cy)的期望种子数量

#init
Ep=np.zeros(ln,Max_t)   #环境参数，由温度、降水、光照等综合评定
Wind_prb=np.zeros(MAX,MAX,Max_t)    #风模型，由数据转化成坐标形式
Gp=np.array([])  #Geographical probability，随机生成的土地适宜程度参数
for i in range(1,MAX+1):
    row_Gp=np.array([])
    for j in range(1,MAX+1):
        np.append(row_Gp,init(np.random.rand()))       #初始化土地适宜程度
    np.append(Gp,row_Gp)

ori_x=MAX//2
ori_y=MAX//2

Grow=np.zeros(MAX,MAX,ln) #ln->根据蒲公英性质确定,生长阶段

#main
for t in range(1,Max_t+1):  #复杂度*O(t)
    for i in range(low_bound,up_bound+1):
        for j in range(lb,rb):       #复杂度*O(Total Area)
            sd_new=0
            for k in range(ln-1,-1,-1):           #此循环更新蒲公英生长状态
                Grow[i][j][k+1]=Grow[i][j][k]
            for k in range(0,len(rel_pos)+1):     #复杂度*O(relative Area)
                Grow[i][j][0]+=sigma_p(rel_pos[k][0],rel_pos[k][1],i,j,t)
            for k in range(0,ln+1):               #实现生长状况
                Grow[i][j][k]*=Ep[ln][t]
            #本段可以提取当前阶段数据，做可视化

#目前待解决的问题：
#1.确定蒲公英生长周期，就可以决定t的一个间隔代表实际多长时间
#2.确定土地适宜程度的随机生成方案，或者找到一块样本土地的数据，确定Gp
#3.需要由风速、风向评估出传播期望参数，确定Wind_prb
#4.需要由降水、温度、光照综合评定环境参数Ep



            
        
        
        
        

In [3]:
import copy


class Cell:
    # 单元格左上角坐标 x 值
    x: float
    # 单元格左上角坐标 y 值
    y: float
    # 单元格高度
    height: float
    # 单元格宽度
    width: float
    # 单元格值，默认为 0
    value: int

    def __init__(self, x: float, y: float, width: float, height: float):
        self.x = x
        self.y = y
        self.height = height
        self.width = width
        self.value = 0


class Box:
    # 窗口宽
    window_width: int
    # 窗口高
    window_height: int
    # 行数
    row_num: int
    # 列数
    col_num: int
    cells: [[Cell]]

    def __init__(self, row_num: int, col_num: int, win_width: int, win_height: int):
        self.window_height = win_height
        self.window_width = win_width
        self.row_num = row_num
        self.col_num = col_num
        self.cells: [[Cell]] = []
        for row in range(0, self.row_num):
            tmp: [Cell] = []
            for col in range(0, self.col_num):
                cell = Cell(
                    row * (self.window_width / self.row_num),
                    col * (self.window_height / self.col_num),
                    self.window_width / self.row_num,
                    self.window_height / self.col_num,
                )
                tmp.append(cell)
            self.cells.append(tmp)
        return

    # 获取指定的 cell
    # row、col 允许为负值，负值时将根据地图的连接进行计算得到正值
    def get_cell(self, row: int, col: int) -> Cell:
        if row < 0:
            row = row % self.row_num - 1
        if row >= self.row_num:
            row = row % self.row_num
        if col < 0:
            col = col % self.col_num - 1
        if col >= self.col_num:
            col = col % self.col_num
        return self.cells[row][col]

    # 获取所有单元
    def get_all_cells(self) -> [Cell]:
        cells = []
        for rows in self.cells:
            for cell in rows:
                cells.append(cell)
        return cells

    # 根据自动机规则，刷新状态
    # 返回状态变更了的单元
    def flush(self) -> [Cell]:
        old_box = copy.deepcopy(self)
        # 需要变更状态的单元
        update_cells_index = []
        # 遍历所有单元，更新值
        for row in range(0, self.row_num):
            for col in range(0, self.col_num):
                # 读取周围8个单元值
                cur = old_box.get_cell(row, col)
                top = old_box.get_cell(row - 1, col)
                left = old_box.get_cell(row, col - 1)
                bottom = old_box.get_cell(row + 1, col)
                right = old_box.get_cell(row, col + 1)
                top_left = old_box.get_cell(row - 1, col - 1)
                bottom_left = old_box.get_cell(row + 1, col - 1)
                top_right = old_box.get_cell(row - 1, col + 1)
                bottom_right = old_box.get_cell(row + 1, col + 1)
                # 存活的节点数量
                surviving_count = top.value \
                                  + left.value \
                                  + bottom.value \
                                  + right.value \
                                  + top_left.value \
                                  + bottom_left.value \
                                  + top_right.value \
                                  + bottom_right.value
                if cur.value == 1:
                    if surviving_count < 2:
                        # 生命稀少，挂了
                        update_cells_index.append([row, col])
                    if surviving_count > 3:
                        # 生命数量过多，挂了
                        update_cells_index.append([row, col])
                if cur.value == 0:
                    if surviving_count == 3:
                        # 生命繁衍
                        update_cells_index.append([row, col])
        update_cells = []
        # 执行状态值变更
        for index in update_cells_index:
            if self.cells[index[0]][index[1]].value == 1:
                self.cells[index[0]][index[1]].value = 0
            else:
                self.cells[index[0]][index[1]].value = 1
            update_cells.append(self.cells[index[0]][index[1]])
        return update_cells

In [4]:
import pygame
import random
import sys
from pygame.locals import *

import box

# 定义窗口宽高
win_width = 800
win_height = 800
row_cell_num = 160
# 初始状态活的单元数量
life_num = 4800


# 绘图
def draw(win, cells: [box.Cell]):
    for cell in cells:
        if cell.value == 0:
            color = (255, 255, 255)
        else:
            color = (0, 0, 0)
        pygame.draw.rect(win, color, pygame.Rect(
            cell.x,
            cell.y,
            cell.width,
            cell.height,
        ))


def run():
    # 是否完成了初始化绘图
    init = False
    # 控制暂停
    change_start = True
    pygame.init()
    # 初始化容器
    box_control = box.Box(row_cell_num, row_cell_num, win_width, win_height)
    # 初始化活的单元
    if life_num > 0:
        for i in range(0, life_num):
            row, col = random.randint(0, box_control.row_num - 1), random.randint(0, box_control.col_num - 1)
            box_control.cells[row][col].value = 1

    # 初始化窗口
    win = pygame.display.set_mode((win_width, win_height))
    pygame.display.set_caption('Cellular Automata')
    clock = pygame.time.Clock()

    while True:
        clock.tick(1)
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN and event.key == K_SPACE:
                # 支持按空格暂停/恢复状态变更
                change_start = not change_start
        if not init:
            # 初始化
            init = True
            all_cells = box_control.get_all_cells()
            draw(win, all_cells)
        else:
            if change_start:
                update_cells = box_control.flush()
                draw(win, update_cells)
        pygame.display.update()


if __name__ == '__main__':
    # 启动程序
    run()

pygame 2.5.2 (SDL 2.28.3, Python 3.11.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


ModuleNotFoundError: No module named 'box'